#1.Setup

In [1]:
!pip install -q -U "tensorflow-text==2.11.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requir

In [2]:
!pip install -q tensorflow_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorflow 2.11.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [3]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [ ]:
tf.get_logger().setLevel('ERROR')
pwd = pathlib.Path.cwd()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/Grammatical_Error_Correction/

/content/drive/MyDrive/Grammatical_Error_Correction


#2.Data Loading

In [6]:
CSV_PATH = './Data/csv_data/c4_200m/data.csv'

In [11]:
df = pd.read_csv(CSV_PATH)

In [12]:
df.head()

,input,target
0,The effect of widespread dud targets two face ...,The effect of widespread dud targets two face ...
1,tax on sales of stores for non residents are s...,Capital Gains tax on the sale of properties fo...
2,Much many brands and sellers still in the market,Many brands and sellers still in the market
3,this is is the latest Maintenance release of S...,This is is the latest maintenance release of S...
4,Fairy Or Not I am the Godmother no just look b...,Fairy Or Not I am the Godmother Not just a loo...


In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((df['input'].values, df['target'].values))

In [9]:
# Train Dataset Example
for input, target in train_dataset.batch(1).take(1):
    for inp in input.numpy():
        print("Incorrect Sentence:", inp.decode('utf-8'))

    for tgt in target.numpy():
        print("Correct Sentence  :", tgt.decode('utf-8'))
    print()

Incorrect Sentence: The effect of widespread dud targets two face up attack position monsters on the field
Correct Sentence  : The effect of widespread dud targets two face up attack position monsters on the field



In [10]:
train_inp = train_dataset.map(lambda inp, tgt: inp)
train_tgt = train_dataset.map(lambda inp, tgt: tgt)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


##2.1.Data Preprocessing

In [13]:
NUM_ROWS = 200000

In [14]:
df = pd.read_csv("/content/drive/MyDrive/C4 200M/C4_200M.tsv-00000-of-00010", delimiter='\t', nrows=NUM_ROWS)
df.columns = ["input", "target"]

In [15]:
df.shape

(200000, 2)

In [16]:
df.head()

,input,target
0,The effect of widespread dud targets two face ...,"1. The effect of ""widespread dud"" targets two ..."
1,tax on sales of stores for non residents are s...,Capital Gains tax on the sale of properties fo...
2,Much many brands and sellers still in the market.,Many brands and sellers still in the market.
3,this is is the latest Maintenance release of S...,This is is the latest maintenance release of S...
4,"Fairy Or Not, I'm the Godmother: no just look,...","Fairy Or Not, I'm the Godmother: Not just a lo..."


In [19]:
df = df.astype(str)

In [20]:
df['input_len'] = [len(text.split()) for text in df.input.values]
df['target_len'] = [len(text.split()) for text in df.target.values]

In [ ]:
# df = df[~(df['input_len'] < 5) & ~(df['input_len'] > 20)]
# df = df[~(df['target_len'] < 5) & ~(df['target_len'] > 20)]

In [21]:
train_dataset = tf.data.Dataset.from_tensor_slices((df['input'].values, df['target'].values))

In [22]:
# Train Dataset Example
for input, target in train_dataset.batch(1).take(1):
    for inp in input.numpy():
        print("Incorrect Sentence:", inp.decode('utf-8'))

    for tgt in target.numpy():
        print("Correct Sentence  :", tgt.decode('utf-8'))
    print()

Incorrect Sentence: The effect of widespread dud targets two face up attack position monsters on the field.
Correct Sentence  : 1. The effect of "widespread dud" targets two face up attack position monsters on the field.



In [23]:
train_inp = train_dataset.map(lambda inp, tgt: inp)
train_tgt = train_dataset.map(lambda inp, tgt: tgt)

#3.Tokenizer

##3.1.Building Vocabulary

In [24]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [25]:
bert_tokenizer_params=dict(lower_case=False)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 80000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [26]:
%%time
inp_vocab = bert_vocab.bert_vocab_from_dataset(
    train_inp.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 22min 23s, sys: 6.33 s, total: 22min 29s
Wall time: 22min 56s


In [27]:
print(inp_vocab[:10])
print(inp_vocab[100:110])
print(inp_vocab[1000:1010])
print(inp_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '#', '$', '%', '&']
['¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«']
['on', 'you', 'that', '##s', 'The', 'it', 'as', 'at', 'by', 'are']
['##👦', '##🔥', '##😀', '##😉', '##😊', '##😍', '##😛', '##😬', '##🙁', '##🙂']


In [28]:
%%time
tgt_vocab = bert_vocab.bert_vocab_from_dataset(
    train_tgt.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 18min 38s, sys: 3.72 s, total: 18min 41s
Wall time: 18min 51s


In [29]:
print(tgt_vocab[:10])
print(tgt_vocab[100:110])
print(tgt_vocab[1000:1010])
print(tgt_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '#', '$', '%', '&']
['£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬']
['you', 'that', 'The', '##s', 'at', 'as', 'are', 'it', 'be', 'by']
['##👦', '##🔥', '##😀', '##😉', '##😊', '##😍', '##😛', '##😬', '##🙁', '##🙂']


In [30]:
def write_vocab_file(filepath, vocab):
    with open(filepath, 'w') as f:
        for token in vocab:
            print(token, file=f)

In [31]:
write_vocab_file('./Data/Tokenizer/inp_vocab.txt', inp_vocab)

In [32]:
write_vocab_file('./Data/Tokenizer/tgt_vocab.txt', tgt_vocab)

In [33]:
!ls ./Data/Tokenizer/*.txt

./Data/Tokenizer/inp_vocab.txt	./Data/Tokenizer/tgt_vocab.txt


##3.2.Building Tokenizer

In [34]:
inp_tokenizer = text.BertTokenizer('./Data/Tokenizer/inp_vocab.txt', **bert_tokenizer_params)
tgt_tokenizer = text.BertTokenizer('./Data/Tokenizer/tgt_vocab.txt', **bert_tokenizer_params)

In [35]:
for inp_examples, tgt_examples in train_dataset.batch(3).take(1):
    for ex in tgt_examples:
        print(ex.numpy())

b'1. The effect of "widespread dud" targets two face up attack position monsters on the field.'
b'Capital Gains tax on the sale of properties for non-residents is set at 21% for 2014 and 20% in 2015 payable on profits earned on the difference of the property value between the year of purchase (purchase price plus costs) and the year of sale (sales price minus costs), based on the approved annual percentage increase on the base value approved by law.'
b'Many brands and sellers still in the market.'


In [36]:
# Tokenize the examples
token_batch = tgt_tokenizer.tokenize(tgt_examples)
token_batch = token_batch.merge_dims(-2,-1)

for ex in token_batch.to_list():
    print(ex)

[20, 17, 1002, 2299, 994, 5, 9731, 13838, 1087, 5, 6715, 1081, 1783, 1030, 3057, 1882, 17867, 998, 991, 1632, 17]
[3987, 16092, 1003, 1739, 998, 991, 1523, 994, 2659, 996, 1435, 16, 2444, 997, 1149, 1004, 1620, 8, 996, 1458, 992, 1207, 8, 995, 1346, 9846, 998, 5670, 3763, 998, 991, 2355, 994, 991, 1398, 1400, 1153, 991, 1065, 994, 1847, 11, 1847, 1354, 2241, 1963, 12, 992, 991, 1065, 994, 1523, 11, 1614, 1354, 9841, 1963, 12, 15, 1155, 998, 991, 2790, 1937, 4751, 1742, 998, 991, 2177, 1400, 2790, 1009, 1521, 17]
[2514, 3415, 992, 9115, 1197, 995, 991, 1278, 17]


In [37]:
# Lookup each token id in the vocabulary.
txt_tokens = tf.gather(tgt_vocab, token_batch)
# Join with spaces.
tf.strings.reduce_join(txt_tokens, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'1 . The effect of " widespread du ##d " targets two face up attack position monsters on the field .',
       b'Capital Gain ##s tax on the sale of properties for non - residents is set at 21 % for 2014 and 20 % in 2015 payable on profits earned on the difference of the property value between the year of purchase ( purchase price plus costs ) and the year of sale ( sales price minus costs ) , based on the approved annual percentage increase on the base value approved by law .',
       b'Many brands and sellers still in the market .'], dtype=object)>

In [38]:
words = inp_tokenizer.detokenize(token_batch)
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'1 . that highly of " Cruz Celebration You " verify work forward their Israel plansmple is \xf0\x9f\x99\x82 Friday .',
       b'adults amplifiers someone is \xf0\x9f\x99\x82 March of wrote in several - remember for find The idea % in five the top % to play Pete is informations snow is \xf0\x9f\x99\x82 garden of \xf0\x9f\x99\x82 14 industry And \xf0\x9f\x99\x82 make of additional ( additional live relationship furniture ) the \xf0\x9f\x99\x82 make of March ( meet live Massage furniture ) , part is \xf0\x9f\x99\x82 Team base impressive road is \xf0\x9f\x99\x82 fresh industry Team are space .',
       b'challenge continues theload think to \xf0\x9f\x99\x82 within .'],
      dtype=object)>

##3.3.Customization and Exporting Tokenizer

###3.3.1.Custom Tokenization

In [39]:
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)

In [40]:
words = tgt_tokenizer.detokenize(add_start_end(token_batch))
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'[START] 1 . The effect of " widespread dud " targets two face up attack position monsters on the field . [END]',
       b'[START] Capital Gains tax on the sale of properties for non - residents is set at 21 % for 2014 and 20 % in 2015 payable on profits earned on the difference of the property value between the year of purchase ( purchase price plus costs ) and the year of sale ( sales price minus costs ) , based on the approved annual percentage increase on the base value approved by law . [END]',
       b'[START] Many brands and sellers still in the market . [END]'],
      dtype=object)>

###3.3.2.Custom Detokenization

In [41]:
def cleanup_text(reserved_tokens, token_txt):
  # Drop the reserved tokens, except for "[UNK]".
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)

  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  # Join them into strings.
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)

  return result

In [42]:
tgt_examples.numpy()

array([b'1. The effect of "widespread dud" targets two face up attack position monsters on the field.',
       b'Capital Gains tax on the sale of properties for non-residents is set at 21% for 2014 and 20% in 2015 payable on profits earned on the difference of the property value between the year of purchase (purchase price plus costs) and the year of sale (sales price minus costs), based on the approved annual percentage increase on the base value approved by law.',
       b'Many brands and sellers still in the market.'], dtype=object)

In [43]:
token_batch = tgt_tokenizer.tokenize(tgt_examples).merge_dims(-2,-1)
words = tgt_tokenizer.detokenize(token_batch)
words

<tf.RaggedTensor [[b'1', b'.', b'The', b'effect', b'of', b'"', b'widespread', b'dud', b'"',
  b'targets', b'two', b'face', b'up', b'attack', b'position', b'monsters',
  b'on', b'the', b'field', b'.']                                          ,
 [b'Capital', b'Gains', b'tax', b'on', b'the', b'sale', b'of',
  b'properties', b'for', b'non', b'-', b'residents', b'is', b'set', b'at',
  b'21', b'%', b'for', b'2014', b'and', b'20', b'%', b'in', b'2015',
  b'payable', b'on', b'profits', b'earned', b'on', b'the', b'difference',
  b'of', b'the', b'property', b'value', b'between', b'the', b'year', b'of',
  b'purchase', b'(', b'purchase', b'price', b'plus', b'costs', b')', b'and',
  b'the', b'year', b'of', b'sale', b'(', b'sales', b'price', b'minus',
  b'costs', b')', b',', b'based', b'on', b'the', b'approved', b'annual',
  b'percentage', b'increase', b'on', b'the', b'base', b'value', b'approved',
  b'by', b'law', b'.']                                                      ,
 [b'Many', b'brands', b'

In [44]:
cleanup_text(reserved_tokens, words).numpy()

array([b'1 . The effect of " widespread dud " targets two face up attack position monsters on the field .',
       b'Capital Gains tax on the sale of properties for non - residents is set at 21 % for 2014 and 20 % in 2015 payable on profits earned on the difference of the property value between the year of purchase ( purchase price plus costs ) and the year of sale ( sales price minus costs ) , based on the approved annual percentage increase on the base value approved by law .',
       b'Many brands and sellers still in the market .'], dtype=object)

###3.3.3.Export

The following code block builds a `CustomTokenizer` class to contain the `text.BertTokenizer` instances, the custom logic, and the `@tf.function` wrappers required for export.

In [45]:
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=False)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create the signatures for export:

    # Include a tokenize signature for a batch of strings.
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))

    # Include `detokenize` and `lookup` signatures for:
    #   * `Tensors` with shapes [tokens] and [batch, tokens]
    #   * `RaggedTensors` with shape [batch, tokens]
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    # These `get_*` methods take no arguments
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()

  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    # Merge the `word` and `word-piece` axes.
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._reserved_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)

Build a `CustomTokenizer` for each language:

In [46]:
tokenizers = tf.Module()
tokenizers.inp = CustomTokenizer(reserved_tokens, './Data/Tokenizer/inp_vocab.txt')
tokenizers.tgt = CustomTokenizer(reserved_tokens, './Data/Tokenizer/tgt_vocab.txt')

Export the tokenizers as a `saved_model`:

In [47]:
model_name = './Data/Tokenizer/c4_200m_tokenizer_v2.0'
tf.saved_model.save(tokenizers, model_name)

Reload the `saved_model` and test the methods:

In [48]:
reloaded_tokenizers = tf.saved_model.load(model_name)
reloaded_tokenizers.inp.get_vocab_size().numpy()

36929

In [49]:
tokens = reloaded_tokenizers.tgt.tokenize(['Hello Parth!'])
tokens.numpy()

array([[   2, 4951, 3426, 1303,    4,    3]])

In [50]:
text_tokens = reloaded_tokenizers.tgt.lookup(tokens)
text_tokens

<tf.RaggedTensor [[b'[START]', b'Hello', b'Part', b'##h', b'!', b'[END]']]>

In [51]:
round_trip = reloaded_tokenizers.tgt.detokenize(tokens)

print(round_trip.numpy()[0].decode('utf-8'))

Hello Parth !


In [52]:
!zip -r {model_name}.zip {model_name}

  adding: Data/Tokenizer/c4_200m_tokenizer_v2.0/ (stored 0%)
  adding: Data/Tokenizer/c4_200m_tokenizer_v2.0/variables/ (stored 0%)
  adding: Data/Tokenizer/c4_200m_tokenizer_v2.0/variables/variables.data-00000-of-00001 (deflated 46%)
  adding: Data/Tokenizer/c4_200m_tokenizer_v2.0/variables/variables.index (deflated 32%)
  adding: Data/Tokenizer/c4_200m_tokenizer_v2.0/assets/ (stored 0%)
  adding: Data/Tokenizer/c4_200m_tokenizer_v2.0/assets/tgt_vocab.txt (deflated 50%)
  adding: Data/Tokenizer/c4_200m_tokenizer_v2.0/assets/inp_vocab.txt (deflated 50%)
  adding: Data/Tokenizer/c4_200m_tokenizer_v2.0/fingerprint.pb (stored 0%)
  adding: Data/Tokenizer/c4_200m_tokenizer_v2.0/saved_model.pb (deflated 91%)


In [53]:
!du -h ./Data/Tokenizer/*.zip

596K	./Data/Tokenizer/c4_200m_tokenizer_v2.0.zip
856K	./Data/Tokenizer/c4_200m_tokenizer.zip
